In [24]:
import urllib.request as req
import bs4    as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

def find_playerdata(Name=""):
    #開啟chrome 並到NBA官網 並搜尋指定球員
    options = Options()
    options.add_argument("--disable-notifications")

    chrome = webdriver.Chrome('./chromedriver')
    chrome.get("https://www.nba.com/players")
    time.sleep(3)

        #接受cookies
    to_accept_cookies = chrome.find_element_by_id("onetrust-accept-btn-handler")
    to_accept_cookies.click()

        #模擬拉動頁面
    for x in range(1, 2):
        chrome.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(2)

        #取得想查詢的球員網址
    root =bs.BeautifulSoup(chrome.page_source,"lxml")
    goto_playerpage = root.find("a",title =" "+Name+" ")["href"]

        #開啟想查詢的球員頁面
    chrome.get("https://www.nba.com"+goto_playerpage)
    time.sleep(2)

    root   = bs.BeautifulSoup(chrome.page_source,"lxml")
    find_playerdata = root.find_all("p",{"class":"nba-player-vitals__top-info-metric"})
    arr=[]

        #抓取球員 身高體重
    for playerdata in find_playerdata:
        playerdata=playerdata.text.replace("/ \n              ", "")
        arr.append(playerdata)

        #抓取球員當季數據
    playerdatas =root.find_all("td")
    for playerdata in playerdatas[0:8]:
        arr.append(playerdata.text)

        #存取球員檔案至指定位置
    playdata = pd.DataFrame({
            "PlayName":[Name], "Height":[arr[0]], "Weight":[arr[1]],
            "MPG"   :[arr[2]], "FG%"   :[arr[3]], "3P%"   :[arr[4]], "FT%"   :[arr[5]],
            "PPG"   :[arr[6]], "RPG"   :[arr[7]], "APG"   :[arr[8]], "BPG"   :[arr[9]],
    })
    return playdata


In [26]:
find_playerdata("LeBron James")

,PlayName,Height,Weight,MPG,FG%,3P%,FT%,PPG,RPG,APG,BPG
0,LeBron James,2.06m,113.4kg,34.9,49.4,34.8,69,25.5,7.9,10.3,0.6
